## Deliverable 2. Create a Customer Travel Destinations Map.

In [62]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [63]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,56.08,79,21,12.35,light rain
1,1,Sabang,ID,5.8933,95.3214,83.12,74,100,15.97,overcast clouds
2,2,Qaanaaq,GL,77.4840,-69.3632,38.07,84,100,5.08,overcast clouds
3,3,Port Alfred,ZA,-33.5906,26.8910,60.91,83,0,15.19,clear sky
4,4,Punta Arenas,CL,-53.1500,-70.9167,39.31,87,20,7.56,few clouds


In [34]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [35]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Sabang,ID,83.12,overcast clouds,5.8933,95.3214,Hotel Citra
1,1,Laguna,US,89.44,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
2,2,Rikitea,PF,75.63,overcast clouds,-23.1203,-134.9692,Pension Maro'i
3,3,Dingle,PH,76.30,light rain,10.9995,122.6711,SEGAYA PROPERTY
4,4,Nikolskoye,RU,73.29,broken clouds,59.7035,30.7861,Tourist House - Sablino
5,5,Jamestown,US,84.09,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
6,6,Butaritari,KI,82.31,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
7,7,Mecca,SA,88.16,scattered clouds,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
8,8,Sao Gabriel Da Cachoeira,BR,75.00,moderate rain,-0.1303,-67.0892,Alarne - Área De Lazer Do Alto Rio Negro
9,9,Conde,BR,83.66,scattered clouds,-7.2597,-34.9075,Pousada Beija Flor


In [36]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                238
City                   238
Country                238
Max Temp               238
Current Description    238
Lat                    238
Lng                    238
Hotel Name             238
dtype: int64

In [37]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                238
City                   238
Country                238
Max Temp               238
Current Description    238
Lat                    238
Lng                    238
Hotel Name             238
dtype: int64

In [38]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sabang,ID,83.12,overcast clouds,5.8933,95.3214,
1,Laguna,US,89.44,clear sky,38.4210,-121.4238,
2,Rikitea,PF,75.63,overcast clouds,-23.1203,-134.9692,
3,Dingle,PH,76.30,light rain,10.9995,122.6711,
4,Nikolskoye,RU,73.29,broken clouds,59.7035,30.7861,
5,Jamestown,US,84.09,clear sky,42.0970,-79.2353,
6,Butaritari,KI,82.31,scattered clouds,3.0707,172.7902,
7,Mecca,SA,88.16,scattered clouds,21.4267,39.8261,
8,Sao Gabriel Da Cachoeira,BR,75.00,moderate rain,-0.1303,-67.0892,
9,Conde,BR,83.66,scattered clouds,-7.2597,-34.9075,


In [39]:
# 6a. Set parameters to search for hotels with 5000 meters.
parameter = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    parameter["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=parameter).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 
     

In [40]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sabang,ID,83.12,overcast clouds,5.8933,95.3214,Hotel Citra
1,Laguna,US,89.44,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
2,Rikitea,PF,75.63,overcast clouds,-23.1203,-134.9692,Pension Maro'i
3,Dingle,PH,76.30,light rain,10.9995,122.6711,SEGAYA PROPERTY
4,Nikolskoye,RU,73.29,broken clouds,59.7035,30.7861,Tourist House - Sablino
5,Jamestown,US,84.09,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
6,Butaritari,KI,82.31,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
7,Mecca,SA,88.16,scattered clouds,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
8,Sao Gabriel Da Cachoeira,BR,75.00,moderate rain,-0.1303,-67.0892,Alarne - Área De Lazer Do Alto Rio Negro
9,Conde,BR,83.66,scattered clouds,-7.2597,-34.9075,Pousada Beija Flor


In [41]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sabang,ID,83.12,overcast clouds,5.8933,95.3214,Hotel Citra
1,Laguna,US,89.44,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
2,Rikitea,PF,75.63,overcast clouds,-23.1203,-134.9692,Pension Maro'i
3,Dingle,PH,76.30,light rain,10.9995,122.6711,SEGAYA PROPERTY
4,Nikolskoye,RU,73.29,broken clouds,59.7035,30.7861,Tourist House - Sablino
5,Jamestown,US,84.09,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
6,Butaritari,KI,82.31,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
7,Mecca,SA,88.16,scattered clouds,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
8,Sao Gabriel Da Cachoeira,BR,75.00,moderate rain,-0.1303,-67.0892,Alarne - Área De Lazer Do Alto Rio Negro
9,Conde,BR,83.66,scattered clouds,-7.2597,-34.9075,Pousada Beija Flor


In [55]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [48]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [46]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))